05) Active Learning 기반 데이터 라벨링
`03_train_and_infer`에서 생성된 `anomaly_scores.csv`와 `02_feature_engineering`에서 생성된 `features_raw.parquet`을 사용합니다.
Active Learning 전략에 따라 **가장 점수가 높거나(P1) 애매한(P2) 데이터**를 우선적으로 보여줍니다.
터미널 입력: `1` (공격/이상), `0` (정상), `s` (건너뛰기), `q` (저장 후 종료)

In [ ]:
import os
import pandas as pd

BASE_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\basemodel"
TARGET_SPLIT = "test" # 라벨링할 데이터셋 (일반적으로 test)

In [ ]:
from common import get_paths, ensure_dir

paths = get_paths(BASE_DIR)

# 1. 필요한 파일 경로 설정
feat_dir = paths[f"data_{TARGET_SPLIT}_feat"]
raw_features_path = os.path.join(feat_dir, "features_raw.parquet")
scores_path = os.path.join(feat_dir, "anomaly_scores.csv")

# 2. 라벨링 결과 저장 경로 설정
labeled_dir = os.path.join(paths['base'], "data", "labeled")
ensure_dir(labeled_dir)
labeled_file_path = os.path.join(labeled_dir, "labeled_data.csv")

print("Raw features:", raw_features_path)
print("Anomaly scores:", scores_path)
print("Labeled data output:", labeled_file_path)

In [ ]:
# 3. 데이터 로드 및 결합
if not (os.path.exists(raw_features_path) and os.path.exists(scores_path)):
    raise FileNotFoundError("Raw features or anomaly scores not found. Run notebooks 02 and 03 first.")

df_raw = pd.read_parquet(raw_features_path)
df_scores = pd.read_csv(scores_path)

df_combined = pd.concat([df_raw, df_scores], axis=1)
print("Combined data shape:", df_combined.shape)

In [ ]:
# 4. Active Learning 우선순위 설정
p95 = df_combined['anomaly_score'].quantile(0.95)
p90 = df_combined['anomaly_score'].quantile(0.90)
p70 = df_combined['anomaly_score'].quantile(0.70)

p1_indices = df_combined[df_combined['anomaly_score'] >= p95].index
p2_indices = df_combined[(df_combined['anomaly_score'] >= p70) & (df_combined['anomaly_score'] < p90)].index

# 중복 제거 및 우선순위에 따라 정렬 (점수 높은 순)
priority_indices = sorted(list(set(p1_indices) | set(p2_indices)), key=lambda x: df_combined.loc[x, 'anomaly_score'], reverse=True)

print(f"P1 (Top 5%): {len(p1_indices)}개")
print(f"P2 (70-90%): {len(p2_indices)}개")
print(f"총 라벨링 대상: {len(priority_indices)}개")

In [ ]:
# 5. 라벨링 루프 실행
# 기존 라벨링 파일이 있으면 불러오기
if os.path.exists(labeled_file_path):
    df_labeled = pd.read_csv(labeled_file_path, index_col=0)
    labeled_indices = set(df_labeled.index)
    records_to_append = []
    print(f"기존 라벨링 데이터 {len(labeled_indices)}건을 불러왔습니다.")
else:
    df_labeled = pd.DataFrame()
    labeled_indices = set()
    records_to_append = []

try:
    # 전체 대상 중 이미 라벨링된 것을 제외하고 진행
    indices_to_label = [idx for idx in priority_indices if idx not in labeled_indices]
    total_count = len(indices_to_label)
    
    for i, idx in enumerate(indices_to_label):
        record = df_combined.loc[idx]
        print("\n--- Labeling Progress: " + f"{i+1}/{total_count} (Index: {idx}) ---")
        # 0이 아닌 값만 출력하여 가독성 향상
        print(record[record != 0].to_string())

        while True:
            label = input("\nEnter label (1: Abnormal, 0: Normal, s: Skip, q: Quit & Save): ")
            if label in ['0', '1', 's', 'q']:
                break
            print("Invalid input. Please enter 0, 1, s, or q.")

        if label == 'q':
            print("Quit requested. Saving progress...")
            break
        if label == 's':
            continue

        # 라벨링 결과를 딕셔너리로 만들고 인덱스 추가
        new_record_dict = record.to_dict()
        new_record_dict['label'] = int(label)
        
        # DataFrame으로 변환 후 인덱스 설정
        new_df_record = pd.DataFrame([new_record_dict], index=[idx])
        records_to_append.append(new_df_record)
        
        # 50건마다 중간 저장
        if len(records_to_append) > 0 and len(records_to_append) % 50 == 0:
            df_labeled = pd.concat([df_labeled] + records_to_append)
            df_labeled.to_csv(labeled_file_path)
            records_to_append = [] # 리스트 비우기
            labeled_indices = set(df_labeled.index) # 저장된 인덱스 업데이트
            print(f"\n*** Progress saved! ({len(df_labeled)} records) ***")

finally:
    # 최종 저장
    if records_to_append:
        df_labeled = pd.concat([df_labeled] + records_to_append)
        df_labeled.to_csv(labeled_file_path)
        print(f"\nFinal data saved to {labeled_file_path} ({len(df_labeled)} records total)")
    else:
        print("\nNo new labels were added.")
